In [4]:
from algoshort import yfinance_handler

In [6]:
from algoshort.yfinance_handler import YFinanceDataHandler
handler = YFinanceDataHandler(
    cache_dir="data_cache",     # e.g., "./data_cache"
    enable_logging=True,        # see progress and warnings
    chunk_size=50,              # batch size for multi-symbol downloads
    log_level=20                # logging.INFO
)


In [7]:
handler.download_data("DVLT", start='2018-01-01', interval="1d")

2025-12-31 11:08:40,263 - algoshort.yfinance_handler.YFinanceDataHandler - INFO - Cache hits: 0, Downloads needed: 1
2025-12-31 11:08:40,263 - INFO - Cache hits: 0, Downloads needed: 1
2025-12-31 11:08:40,263 - algoshort.yfinance_handler.YFinanceDataHandler - INFO - Downloading 1 symbols in 1 chunks of 50
2025-12-31 11:08:40,263 - INFO - Downloading 1 symbols in 1 chunks of 50
2025-12-31 11:08:40,604 - ERROR - Failed to get ticker 'DVLT' reason: Failed to perform, curl: (60) SSL certificate problem: self signed certificate in certificate chain. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
2025-12-31 11:08:40,889 - algoshort.yfinance_handler.YFinanceDataHandler - ERROR - Error downloading chunk 1: Failed to perform, curl: (60) SSL certificate problem: self signed certificate in certificate chain. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
2025-12-31 11:08:40,889 - ERROR - Error downloading chunk 1: Failed to perform, curl: (60)

{}